# Loading data and analyzing 

we will be loading data from various files that describes EHRs and performing some analysis on them.

In [2]:
!rm -rf csv
!wget https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
!unzip synthea_sample_data_csv_apr2020.zip
!mv csv data

--2023-05-05 14:53:58--  https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
Resolving synthetichealth.github.io (synthetichealth.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to synthetichealth.github.io (synthetichealth.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8982431 (8.6M) [application/zip]
Saving to: ‘synthea_sample_data_csv_apr2020.zip.1’

synthea_sample_data 100%[===================>]   8.57M  --.-KB/s    in 0.03s   

2023-05-05 14:53:58 (280 MB/s) - ‘synthea_sample_data_csv_apr2020.zip.1’ saved [8982431/8982431]

Archive:  synthea_sample_data_csv_apr2020.zip
   creating: csv/
  inflating: csv/medications.csv     
  inflating: csv/providers.csv       
  inflating: csv/payer_transitions.csv  
  inflating: csv/imaging_studies.csv  
  inflating: csv/supplies.csv        
  inflating: csv/payers.csv          
  inflating: csv/allergies.csv     

## Required libraries and packages

In [3]:
import pandas as pd
import numpy as np

import glob
import pprint 
import plotly
import plotly.graph_objs as go


import folium
from folium.plugins import MarkerCluster

from itertools import chain
import math
from collections import Counter
from prettytable import PrettyTable


import warnings
warnings.filterwarnings("ignore")

print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
print("plotly version:", plotly.__version__)
print("folium version:", folium.__version__)

Pandas version: 1.5.3
Numpy version: 1.22.4
plotly version: 5.13.1
folium version: 0.14.0


## Dataset structure

```text
├── allergies.csv
├── careplans.csv
├── conditions.csv
├── devices.csv
├── encounters.csv
├── imaging_studies.csv
├── immunizations.csv
├── medications.csv
├── observations.csv
├── organizations.csv
├── patients.csv
├── payer_transitions.csv
├── payers.csv
├── procedures.csv
├── providers.csv
└── supplies.csv
```

In [4]:
for f in glob.glob("data/*.csv"):
    df = pd.read_csv(f)
    print("--", f.split("/")[-1], "COLUMNS:", ", ".join(list(df.columns)).lower())

-- payers.csv COLUMNS: id, name, address, city, state_headquartered, zip, phone, amount_covered, amount_uncovered, revenue, covered_encounters, uncovered_encounters, covered_medications, uncovered_medications, covered_procedures, uncovered_procedures, covered_immunizations, uncovered_immunizations, unique_customers, qols_avg, member_months
-- conditions.csv COLUMNS: start, stop, patient, encounter, code, description
-- allergies.csv COLUMNS: start, stop, patient, encounter, code, description
-- organizations.csv COLUMNS: id, name, address, city, state, zip, lat, lon, phone, revenue, utilization
-- medications.csv COLUMNS: start, stop, patient, payer, encounter, code, description, base_cost, payer_coverage, dispenses, totalcost, reasoncode, reasondescription
-- patients.csv COLUMNS: id, birthdate, deathdate, ssn, drivers, passport, prefix, first, last, suffix, maiden, marital, race, ethnicity, gender, birthplace, address, city, state, county, zip, lat, lon, healthcare_expenses, healthca

# Utilities

In [5]:
def count_missing_values(data):
    """
    Counts the number of missing values in each column of a DataFrame and returns the results as a new DataFrame.
    """
    missing_values = data.isnull().sum()
    missing_df = pd.DataFrame({'column_name': missing_values.index, 'missing_count': missing_values.values})
    return missing_df

In [6]:
def find_top_5_values(data, column):
    """
    Finds the top 5 most frequent values in a column of a DataFrame and returns the results as a new DataFrame.
    """
    top_10_values = data[column].value_counts().nlargest(5).reset_index()
    top_10_values.columns = [column, 'count']
    return top_10_values

In [7]:
def get_top_value_counts(df, group_col, count_col, num_top_values):
    # Get the top values in the group column
    top_values = df[group_col].value_counts().head(num_top_values).index.tolist()

    # Group the data by the group column and count the occurrences of values in the count column
    counts_by_group_and_count = df.groupby(group_col)[count_col].value_counts()

    # Filter the results to only include the top values in the group column
    counts_by_group_and_count = counts_by_group_and_count.loc[top_values, :]

    return counts_by_group_and_count

In [8]:
def get_top_values_with_comparison(df1, df2, group_col, count_col, join_col, compare_col, num_top_values):
    # Get the top values in the group column
    top_values = df1[group_col].value_counts().head(num_top_values).index.tolist()

    # Group the data by the group column and count the occurrences of values in the count column
    counts_by_group_and_count = df1.groupby(group_col)[count_col].value_counts()

    # Filter the results to only include the top values in the group column
    counts_by_group_and_count = counts_by_group_and_count.loc[top_values, :]

    # Join with the other DataFrame on the specified column
    joined_data = counts_by_group_and_count.reset_index(name='count').merge(df2[[join_col, compare_col]], left_on=group_col, right_on=join_col)

    return joined_data

# Patients data

**Analysis**

- Average Age is 46 years old.
- There all 1171 unique pateints.
- Most patient race is white with Ethnicity nonhispanic.
- Most patients are located in Boston.

```bash
+-----------------------------+-------+
| Statistical_Information_Age | Value |
+-----------------------------+-------+
|           Min_age           |   3   |
|            Q1_age           |   25  |
|          Median_age         |   46  |
|            Q3_age           |   66  |
|           Max_age           |  113  |
+-----------------------------+-------+
```

In [9]:
patients = pd.read_csv("data/patients.csv")

# drop columns
columns_to_drop = ["DRIVERS", "PASSPORT", "PREFIX", "MAIDEN", "ZIP", "SUFFIX"]
patients = patients.drop(columns_to_drop, axis=1)

patients["BIRTHDATE"] = pd.to_datetime(patients["BIRTHDATE"])
patients["DEATHDATE"] = pd.to_datetime(patients["DEATHDATE"])

# check for missing values
count_missing_values(patients)

,column_name,missing_count
0,Id,0
1,BIRTHDATE,0
2,DEATHDATE,1000
3,SSN,0
4,FIRST,0
5,LAST,0
6,MARITAL,380
7,RACE,0
8,ETHNICITY,0
9,GENDER,0


In [10]:
# Patient Identifie 
column_to_check = 'Id'
are_values_unique = not patients.duplicated(subset=[column_to_check]).any()
if are_values_unique:
    print(f"All values in column {column_to_check} are unique.")
else:
    print(f"There are duplicate values in column {column_to_check}.")

All values in column Id are unique.


In [11]:
# Social Security Number of Patients
column_to_check = 'SSN'
are_values_unique = not patients.duplicated(subset=[column_to_check]).any()
if are_values_unique:
    print(f"All values in column {column_to_check} are unique.")
else:
    print(f"There are duplicate values in column {column_to_check}.")

There are duplicate values in column SSN.


In [12]:
# two different patients have same SSN. need more contient to check it future 

column_name = 'SSN'
duplicates_ssn = patients[patients.duplicated(subset=[column_name], keep=False)]

# the "999-56-8425" is duplicate in the file 

patients[patients["SSN"] == "999-56-8425"]

,Id,BIRTHDATE,DEATHDATE,SSN,FIRST,LAST,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
304,f25be418-dc57-4c06-855c-b0e6315e4189,1966-03-16,NaT,999-56-8425,Dannielle300,Cormier289,S,white,nonhispanic,F,Springfield Massachusetts US,884 Volkman Glen,Westfield,Massachusetts,Hampden County,42.171060,-72.803338,625541.72,6267.72
902,10134dbf-72d1-4381-b8f3-9530cca6622a,1958-09-08,NaT,999-56-8425,Ali918,Vandervort697,S,white,nonhispanic,M,Marlborough Massachusetts US,392 Jones Haven Suite 29,Salem,Massachusetts,Essex County,42.452581,-70.850422,1645897.89,4497.16


In [13]:
# calculate age based on the today date

import datetime

# Define a lambda function to extract year, month, and day from a date
extract_year = lambda x: x.year
extract_month = lambda x: x.month
extract_day = lambda x: x.day

# Apply the lambda functions to the birth date column to create new columns
patients['birth_year'] = patients['BIRTHDATE'].apply(extract_year)
patients['birth_month'] = patients['BIRTHDATE'].apply(extract_month)
patients['birth_day'] = patients['BIRTHDATE'].apply(extract_day)


def calculate_age(row):
    birth_date = datetime.date(row['birth_year'], row['birth_month'], row['birth_day'])
    today = datetime.date.today()
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age

# Apply the function to the DataFrame to create a new column
patients['AGE'] = patients.apply(calculate_age, axis=1)

In [14]:
# drop columns
columns_to_drop = ["birth_year", "birth_month", "birth_day"]
patients = patients.drop(columns_to_drop, axis=1)

In [15]:
# Gender distribution

gender_counts = patients['GENDER'].value_counts()

# Define the colors for the pie chart
colors = ['#FFA07A', '#6495ED']

# Create a pie chart of the gender counts with custom colors
fig = go.Figure(data=[go.Pie(labels=gender_counts.index, 
                              values=gender_counts.values,
                              marker=dict(colors=colors))])

# Add a title to the plot
fig.update_layout(title_text='Gender Distribution')

# Display the plot
fig.show()

In [16]:
# Ethnicity distribution


# Count the number of occurrences of each ethnicity
ethnicity_counts = patients['ETHNICITY'].value_counts()

# Define the colors for the pie chart
colors = ['#FFA07A', '#6495ED']

# Create a pie chart of the ethnicity counts with custom colors
fig = go.Figure(data=[go.Pie(labels=ethnicity_counts.index, 
                              values=ethnicity_counts.values,
                              marker=dict(colors=colors))])

# Add a title to the plot
fig.update_layout(title_text='Ethnicity Distribution')

# Display the plot
fig.show()

In [17]:
# Count the number of occurrences of each gender
race_counts = patients['RACE'].value_counts()

# Define the colors for the pie chart
colors = ['#FFA07A', '#6495ED', "#228B22", "#FFD700", "#9400D3" ]

# Create a pie chart of the gender counts with custom colors
fig = go.Figure(data=[go.Pie(labels=race_counts.index, 
                              values=race_counts.values,
                              marker=dict(colors=colors))])

# Add a title to the plot
fig.update_layout(title_text='Race Distribution')

# Display the plot
fig.show()

In [18]:
# Age distribution

import plotly.express as px
fig = px.histogram(patients, x="AGE", nbins=200, marginal="box", color_discrete_sequence=["#6495ED"])

# Customize the layout
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    font=dict(
        family="Arial",
        size=14,
        color="#6495ED"
    ),
    bargap=0.1
)

fig.show()

In [19]:
# Patient Location

# Define the latitude and longitude data
latitudes = patients["LAT"].values
longitudes = patients["LON"].values

# Create a folium map centered on the first latitude and longitude in the data
map_center = [latitudes[0], longitudes[0]]
my_map = folium.Map(location=map_center, zoom_start=7.5)

marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map for each latitude and longitude in the data
for lat, lon in zip(latitudes, longitudes):
    marker = folium.Marker(location=[lat, lon])
    marker.add_to(marker_cluster)

# Show the map
my_map

# Allergies data

**Analysis**

There are a relation between allergies file with patients and encounters file.

- As the allergies may not be treatable, it is reasonable that the column "STOP" contains missing values.
- Allergy to mold (or mould) is highest type of allergy between patients. It is a type of allergic reaction that occurs when the immune system overreacts to the presence of mold in the environment and their immune system produces antibodies to fight off what it perceives as a threat. This immune response can cause symptoms such as sneezing, runny nose, itchy eyes, and skin rash. In severe cases, mold allergies can trigger asthma attacks.

In [20]:
allergies = pd.read_csv("data/allergies.csv")
allergies.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1982-10-25,NaN,76982e06-f8b8-4509-9ca3-65a99c8650fe,b896bf40-8b72-42b7-b205-142ee3a56b55,300916003,Latex allergy


In [21]:
count_missing_values(allergies)

,column_name,missing_count
0,START,0
1,STOP,533
2,PATIENT,0
3,ENCOUNTER,0
4,CODE,0
5,DESCRIPTION,0


In [22]:
find_top_5_values(allergies, "DESCRIPTION")

,DESCRIPTION,count
0,Allergy to mould,81
1,Dander (animal) allergy,77
2,Allergy to tree pollen,63
3,House dust mite allergy,61
4,Allergy to grass pollen,60


# Careplans data

- There is a relation between careplans with patients and encounters file.
- Most patient with Acute bronchitis (disorder) recieved careplans.

In [23]:
careplans = pd.read_csv("data/careplans.csv")
careplans.head(1)

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)


In [24]:
count_missing_values(careplans)

,column_name,missing_count
0,Id,0
1,START,0
2,STOP,1532
3,PATIENT,0
4,ENCOUNTER,0
5,CODE,0
6,DESCRIPTION,0
7,REASONCODE,327
8,REASONDESCRIPTION,327


In [25]:
find_top_5_values(careplans, "DESCRIPTION")

,DESCRIPTION,count
0,Respiratory therapy,633
1,Routine antenatal care,397
2,Diabetes self management plan,339
3,Lifestyle education regarding hypertension,302
4,Physical therapy procedure,230


In [26]:
find_top_5_values(careplans, "REASONDESCRIPTION")

,REASONDESCRIPTION,count
0,Acute bronchitis (disorder),633
1,Normal pregnancy,397
2,Hypertension,302
3,Prediabetes,295
4,Hyperlipidemia,136


# Conditions data

- The file contains information related to the patient conditon.
- Top condition between the patient is Viral sinusitis (disorder).

In [27]:
conditions = pd.read_csv("data/conditions.csv")
conditions.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,NaN,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)


In [28]:
count_missing_values(conditions)

,column_name,missing_count
0,START,0
1,STOP,3811
2,PATIENT,0
3,ENCOUNTER,0
4,CODE,0
5,DESCRIPTION,0


In [29]:
find_top_5_values(conditions, "DESCRIPTION")

,DESCRIPTION,count
0,Viral sinusitis (disorder),1248
1,Acute viral pharyngitis (disorder),653
2,Acute bronchitis (disorder),563
3,Normal pregnancy,516
4,Body mass index 30+ - obesity (finding),449


# Devies data

- Top device between the patient is 'Implantable defibrillator  device (physical object)'
- An implantable defibrillator device is a medical device that is surgically implanted under the skin of a person's chest to help regulate abnormal heart rhythms. It is also commonly known as an implantable cardioverter defibrillator (ICD).

In [30]:
# payers, encounter
devices = pd.read_csv("data/devices.csv")
devices.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,UDI
0,2001-07-04T08:42:44Z,NaN,d49f748f-928d-40e8-92c8-73e4c5679711,2500b8bd-dc98-44ef-a252-22dc4f81d61b,72506001,Implantable defibrillator device (physical ob...,(01)67677988606464(11)010613(17)260628(10)2882...


In [31]:
count_missing_values(devices)

,column_name,missing_count
0,START,0
1,STOP,78
2,PATIENT,0
3,ENCOUNTER,0
4,CODE,0
5,DESCRIPTION,0
6,UDI,0


In [32]:
find_top_5_values(devices, "DESCRIPTION")

,DESCRIPTION,count
0,Implantable defibrillator device (physical ob...,47
1,Coronary artery stent (physical object),21
2,Implantable cardiac pacemaker (physical object),10


# Encounter data

Top five encounter types are:

- **Wellness:** checkup, Medical routines.
- **Ambulatory:** The term "ambulatory" refers to the ability to walk or move around. In a medical context, it typically means that a patient is able to walk and move around without assistance or support.
- **Outpatient:** In a medical context, an "outpatient" is a patient who receives medical care without being admitted to a hospital or other healthcare facility for an overnight stay.
-  **Urgentcare:** Immediate medical attention to patients with non-life-threatening conditions.
-  **Emergency:** life-threatening injuries or conditions that require immediate attention.

In [33]:
# payers, organization, provider, payer
encounters = pd.read_csv("data/encounters.csv")
encounters.head(1)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,185345009,Encounter for symptom,129.16,129.16,54.16,10509002.0,Acute bronchitis (disorder)


In [34]:
count_missing_values(encounters)

,column_name,missing_count
0,Id,0
1,START,0
2,STOP,0
3,PATIENT,0
4,ORGANIZATION,0
5,PROVIDER,0
6,PAYER,0
7,ENCOUNTERCLASS,0
8,CODE,0
9,DESCRIPTION,0


In [35]:
find_top_5_values(encounters, "ENCOUNTERCLASS")

,ENCOUNTERCLASS,count
0,wellness,19106
1,ambulatory,18936
2,outpatient,9003
3,urgentcare,2373
4,emergency,2090


In [36]:
find_top_5_values(encounters, "DESCRIPTION")

,DESCRIPTION,count
0,General examination of patient (procedure),14946
1,Encounter for problem (procedure),5196
2,Encounter for check up (procedure),4515
3,Well child visit (procedure),4144
4,Encounter for symptom,3929


In [37]:
get_top_value_counts(encounters, "ENCOUNTERCLASS", "DESCRIPTION", 2)

ENCOUNTERCLASS  DESCRIPTION                                                
wellness        General examination of patient (procedure)                     14946
                Well child visit (procedure)                                    4144
                Death Certification                                               16
ambulatory      Encounter for problem (procedure)                               3877
                Encounter for symptom                                           3732
                Prenatal visit                                                  2676
                Follow-up encounter                                             2282
                Outpatient procedure                                            1543
                Encounter for problem                                           1389
                Encounter for 'check-up'                                         893
                Prenatal initial visit                                    

# Imaging studies data

In a medical context, imaging studies refer to diagnostic tests that produce images of the inside of the body. Top 5 are:
- CT Image Storage	
- Digital X-Ray Image Storage	
- Ultrasound Multiframe Image Storage	
- Digital X-Ray Image Storage – for Presentation	
- Ultrasound Image Storage

In [38]:
imaging_studies = pd.read_csv("data/imaging_studies.csv")
imaging_studies.head(1)

,Id,DATE,PATIENT,ENCOUNTER,BODYSITE_CODE,BODYSITE_DESCRIPTION,MODALITY_CODE,MODALITY_DESCRIPTION,SOP_CODE,SOP_DESCRIPTION
0,d3e49b38-7634-4416-879d-7bc68bf3e7df,2014-07-08T15:35:36Z,b58731cc-2d8b-4c2d-b327-4cab771af3ef,3a36836d-da25-4e73-808b-972b669b7e4e,40983000,Arm,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage


In [39]:
count_missing_values(imaging_studies)

,column_name,missing_count
0,Id,0
1,DATE,0
2,PATIENT,0
3,ENCOUNTER,0
4,BODYSITE_CODE,0
5,BODYSITE_DESCRIPTION,0
6,MODALITY_CODE,0
7,MODALITY_DESCRIPTION,0
8,SOP_CODE,0
9,SOP_DESCRIPTION,0


In [40]:
find_top_5_values(imaging_studies, "MODALITY_DESCRIPTION")

,MODALITY_DESCRIPTION,count
0,Computed Tomography,328
1,Digital Radiography,278
2,Ultrasound,173
3,Computed Radiography,76


In [41]:
find_top_5_values(imaging_studies, "SOP_DESCRIPTION")

,SOP_DESCRIPTION,count
0,CT Image Storage,328
1,Digital X-Ray Image Storage,217
2,Ultrasound Multiframe Image Storage,172
3,Digital X-Ray Image Storage – for Presentation,137
4,Ultrasound Image Storage,1


# Immunization data

In a medical context, immunizations refer to the process of receiving a vaccine to protect against infectious diseases. Immunization works by stimulating the immune system to produce antibodies that can recognize and attack specific viruses or bacteria. 

Top 5 are:

- Influenza seasonal injectable preservative 
- Td (adult) preservative free	
- Pneumococcal conjugate PCV 13	
- DTaP	
- IPV

In [42]:
immunizations = pd.read_csv("data/immunizations.csv")
immunizations.head(1)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST
0,2010-07-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,140,Influenza seasonal injectable preservative ...,140.52


In [43]:
count_missing_values(immunizations)

,column_name,missing_count
0,DATE,0
1,PATIENT,0
2,ENCOUNTER,0
3,CODE,0
4,DESCRIPTION,0
5,BASE_COST,0


In [44]:
find_top_5_values(immunizations, "DESCRIPTION")

,DESCRIPTION,count
0,Influenza seasonal injectable preservative ...,9385
1,Td (adult) preservative free,830
2,Pneumococcal conjugate PCV 13,661
3,DTaP,572
4,IPV,481


In [45]:
get_top_value_counts(immunizations, "DESCRIPTION", "BASE_COST", 5)

DESCRIPTION                                         BASE_COST
Influenza  seasonal  injectable  preservative free  140.52       9385
Td (adult) preservative free                        140.52        830
Pneumococcal conjugate PCV 13                       140.52        661
DTaP                                                140.52        572
IPV                                                 140.52        481
Name: BASE_COST, dtype: int64

# Medications data
- We could see in some cased patients do not have insurance and payer does not coergae the total cost of the medication

In [46]:
medications = pd.read_csv("data/medications.csv")
medications.head(1)

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2010-05-05T00:26:23Z,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,Etonogestrel 68 MG Drug Implant,677.08,0.0,12,8124.96,NaN,NaN


In [47]:
count_missing_values(medications)

,column_name,missing_count
0,START,0
1,STOP,1895
2,PATIENT,0
3,PAYER,0
4,ENCOUNTER,0
5,CODE,0
6,DESCRIPTION,0
7,BASE_COST,0
8,PAYER_COVERAGE,0
9,DISPENSES,0


In [48]:
find_top_5_values(medications, "DESCRIPTION")

,DESCRIPTION,count
0,Hydrochlorothiazide 25 MG Oral Tablet,3954
1,insulin human isophane 70 UNT/ML / Regular In...,3880
2,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen],3388
3,Atenolol 50 MG / Chlorthalidone 25 MG Oral Tablet,3347
4,24 HR Metformin hydrochloride 500 MG Extended ...,2895


# Observation data

During patient encounter. the top obervation were:

- Pain severity - 0-10 verbal numeric rating 
- Diastolic Blood Pressure	
- Systolic Blood Pressure	
- Body Height	
- Tobacco smoking status NHIS

In [49]:
observations = pd.read_csv("data/observations.csv")
observations.head(1)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric


In [50]:
count_missing_values(observations)

,column_name,missing_count
0,DATE,0
1,PATIENT,0
2,ENCOUNTER,30363
3,CODE,0
4,DESCRIPTION,0
5,VALUE,0
6,UNITS,12735
7,TYPE,0


In [51]:
find_top_5_values(observations, "DESCRIPTION")

,DESCRIPTION,count
0,Pain severity - 0-10 verbal numeric rating [Sc...,16820
1,Diastolic Blood Pressure,12963
2,Systolic Blood Pressure,12963
3,Body Height,12552
4,Tobacco smoking status NHIS,12552


# Organization data

> "utilization" typically refers to the amount or frequency of use of hospital resources or services, such as hospital beds, equipment, procedures, or staff time. 

In [52]:
organizations = pd.read_csv("data/organizations.csv")
organizations.head(1)

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
0,ef58ea08-d883-3957-8300-150554edc8fb,HEALTHALLIANCE HOSPITALS INC,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876,9784662000,198002.28,1557


In [53]:
count_missing_values(organizations)

,column_name,missing_count
0,Id,0
1,NAME,0
2,ADDRESS,0
3,CITY,0
4,STATE,0
5,ZIP,0
6,LAT,0
7,LON,0
8,PHONE,184
9,REVENUE,0


In [54]:
find_top_5_values(organizations, "NAME")

,NAME,count
0,MINUTECLINIC DIAGNOSTIC OF MASSACHUSETTS LLC,13
1,UMASS MEMORIAL MEDICAL GROUP INC.,12
2,MEDICAL CARE OF BOSTON MANAGEMENT CORPORATION,9
3,STEWARD MEDICAL GROUP INC,7
4,SOUTHCOAST PHYSICIANS GROUP INC..,7


In [55]:
get_top_value_counts(organizations, "NAME", "UTILIZATION", 1)

NAME                                           UTILIZATION
MINUTECLINIC DIAGNOSTIC OF MASSACHUSETTS  LLC  65             2
                                               14             1
                                               15             1
                                               16             1
                                               24             1
                                               31             1
                                               36             1
                                               37             1
                                               55             1
                                               56             1
                                               63             1
                                               112            1
Name: UTILIZATION, dtype: int64

# Payer transitions data

The Payer represent the insurance of the patient. Most of patient done the patient by themselves.

- Self
- Guardian	
- Spouse

In [56]:
payer_transitions = pd.read_csv("data/payer_transitions.csv")
payer_transitions.head(1)

,PATIENT,START_YEAR,END_YEAR,PAYER,OWNERSHIP
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989,1998,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,Guardian


In [57]:
count_missing_values(payer_transitions)

,column_name,missing_count
0,PATIENT,0
1,START_YEAR,0
2,END_YEAR,0
3,PAYER,0
4,OWNERSHIP,236


In [58]:
find_top_5_values(payer_transitions, "OWNERSHIP")

,OWNERSHIP,count
0,Self,2299
1,Guardian,779
2,Spouse,487


# Payers data 

The top three insurance companies between the patients are:

- Dual Eligible	
- Medicare	
- Medicaid

In [59]:
payers = pd.read_csv("data/payers.csv")
payers.head(1)

,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,REVENUE,...,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323,141676.87,119449.83,1305000.0,...,0,556,0,280,0,223,0,25,0.36281,3348


In [60]:
count_missing_values(payers)

,column_name,missing_count
0,Id,0
1,NAME,0
2,ADDRESS,1
3,CITY,1
4,STATE_HEADQUARTERED,1
5,ZIP,1
6,PHONE,1
7,AMOUNT_COVERED,0
8,AMOUNT_UNCOVERED,0
9,REVENUE,0


In [61]:
find_top_5_values(payers, "NAME")

,NAME,count
0,Dual Eligible,1
1,Medicare,1
2,Medicaid,1
3,Humana,1
4,Blue Cross Blue Shield,1


# Procedure data

In [62]:
procedures = pd.read_csv("data/procedures.csv")
procedures.head(1)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,169553002,Insertion of subcutaneous contraceptive,14896.56,NaN,NaN


In [63]:
count_missing_values(procedures)

,column_name,missing_count
0,DATE,0
1,PATIENT,0
2,ENCOUNTER,0
3,CODE,0
4,DESCRIPTION,0
5,BASE_COST,0
6,REASONCODE,15544
7,REASONDESCRIPTION,15544


In [64]:
find_top_5_values(procedures, "DESCRIPTION")

,DESCRIPTION,count
0,Medication Reconciliation (procedure),5632
1,Renal dialysis (procedure),3389
2,Auscultation of the fetal heart,2705
3,Evaluation of uterine fundal height,2705
4,Subcutaneous immunotherapy,1497


# Provider data

> The number of female speciality is greated than Male.

In [65]:
providers = pd.read_csv("data/providers.csv")
providers.head(1)

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON,UTILIZATION
0,3421aa75-dec7-378d-a9e0-0bc764e4cb0d,ef58ea08-d883-3957-8300-150554edc8fb,Tomas436 Sauer652,M,GENERAL PRACTICE,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876,1557


In [66]:
count_missing_values(providers)

,column_name,missing_count
0,Id,0
1,ORGANIZATION,0
2,NAME,0
3,GENDER,0
4,SPECIALITY,0
5,ADDRESS,0
6,CITY,0
7,STATE,0
8,ZIP,0
9,LAT,0


In [67]:
find_top_5_values(providers, "SPECIALITY")

,SPECIALITY,count
0,GENERAL PRACTICE,1120
1,INTERNAL MEDICINE,608
2,NURSE PRACTITIONER,511
3,CLINICAL SOCIAL WORKER,367
4,PHYSICIAN ASSISTANT,311


In [68]:
get_top_value_counts(providers, "SPECIALITY", "GENDER", 3)

SPECIALITY          GENDER
GENERAL PRACTICE    F         572
                    M         548
INTERNAL MEDICINE   F         328
                    M         280
NURSE PRACTITIONER  F         278
                    M         233
Name: GENDER, dtype: int64

# Relational dataset

The final data are organized based on relationships among the other datasets. 

In [69]:
data = pd.DataFrame([])

# merge patients with encounters
data = pd.merge(patients.add_prefix('PATIENT_'), encounters.add_prefix('ENCOUNTER_'), how='right', left_on='PATIENT_Id', right_on='ENCOUNTER_PATIENT')
data = data.drop("ENCOUNTER_PATIENT", axis=1)


# merge data with organizations
data = pd.merge(data, organizations.add_prefix('ORGANIZATION_'), how='left', left_on='ENCOUNTER_ORGANIZATION', right_on='ORGANIZATION_Id')
data = data.drop("ENCOUNTER_ORGANIZATION", axis=1)


# merge data with providers
data = pd.merge(data, providers.add_prefix('PROVIDER_'), how='left',  left_on='ENCOUNTER_PROVIDER', right_on='PROVIDER_Id')
data = data.drop("ENCOUNTER_PROVIDER", axis=1)
data = data.drop("PROVIDER_ORGANIZATION", axis=1)


# merge data with payers
data = pd.merge(data, payers.add_prefix('PAYER_'), how='left', left_on='ENCOUNTER_PAYER', right_on='PAYER_Id')
data = data.drop("ENCOUNTER_PAYER", axis=1)


# merge data with observations
group_by = ['PATIENT', 'ENCOUNTER']
observations_grouped = observations.groupby(group_by).agg({k: list for k in observations.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, observations_grouped.add_prefix('OBS_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['OBS_PATIENT', 'OBS_ENCOUNTER'])
data = data.drop("OBS_PATIENT", axis=1)
data = data.drop("OBS_ENCOUNTER", axis=1)


# merge data with conditions
group_by = ['PATIENT', 'ENCOUNTER']
conditions_grouped = conditions.groupby(group_by).agg({k: list for k in conditions.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, conditions_grouped.add_prefix('COND_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['COND_PATIENT', 'COND_ENCOUNTER'])
data = data.drop("COND_PATIENT", axis=1)
data = data.drop("COND_ENCOUNTER", axis=1)


# merge data with medications
group_by = ['PATIENT', 'PAYER', 'ENCOUNTER']
medications_grouped = medications.groupby(group_by).agg({k: list for k in medications.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, medications_grouped.add_prefix('MEDICATION_'), how='left', left_on=['PATIENT_Id', 'PAYER_Id', 'ENCOUNTER_Id'], right_on=['MEDICATION_PATIENT', 'MEDICATION_PAYER', 'MEDICATION_ENCOUNTER'])
data = data.drop("MEDICATION_PATIENT", axis=1)
data = data.drop("MEDICATION_PAYER", axis=1)
data = data.drop("MEDICATION_ENCOUNTER", axis=1)


# merge data with imaging_studies
group_by = ['PATIENT', 'ENCOUNTER']
imaging_studies_grouped = imaging_studies.groupby(group_by).agg({k: list for k in imaging_studies.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, imaging_studies_grouped.add_prefix('IMG_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['IMG_PATIENT', 'IMG_ENCOUNTER'])
data = data.drop("IMG_PATIENT", axis=1)
data = data.drop("IMG_ENCOUNTER", axis=1)


# merge data with procedures
group_by = ['PATIENT', 'ENCOUNTER']
procedures_grouped = procedures.groupby(group_by).agg({k: list for k in procedures.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, procedures_grouped.add_prefix('PROC_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['PROC_PATIENT', 'PROC_ENCOUNTER'])
data = data.drop("PROC_PATIENT", axis=1)
data = data.drop("PROC_ENCOUNTER", axis=1)


# merge data with careplans
group_by = ['PATIENT', 'ENCOUNTER']
careplans_grouped = careplans.groupby(group_by).agg({k: list for k in careplans.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, careplans_grouped.add_prefix('CAREPLAN_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['CAREPLAN_PATIENT', 'CAREPLAN_ENCOUNTER'])
data = data.drop("CAREPLAN_PATIENT", axis=1)
data = data.drop("CAREPLAN_ENCOUNTER", axis=1)


# merge data with allergies
group_by = ['PATIENT', 'ENCOUNTER']
allergies_grouped = allergies.groupby(group_by).agg({k: list for k in allergies.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, allergies_grouped.add_prefix('ALLERGY_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['ALLERGY_PATIENT', 'ALLERGY_ENCOUNTER'])
data = data.drop("ALLERGY_PATIENT", axis=1)
data = data.drop("ALLERGY_ENCOUNTER", axis=1)


# merge data with devices
group_by = ['PATIENT', 'ENCOUNTER']
devices_grouped = devices.groupby(group_by).agg({k: list for k in devices.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, devices_grouped.add_prefix('DEVICE_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['DEVICE_PATIENT', 'DEVICE_ENCOUNTER'])
data = data.drop("DEVICE_PATIENT", axis=1)
data = data.drop("DEVICE_ENCOUNTER", axis=1)


# merge data with immunizations
group_by = ['PATIENT', 'ENCOUNTER']
immunizations_grouped = immunizations.groupby(group_by).agg({k: list for k in immunizations.columns.tolist() if k not in group_by}).reset_index()
data = pd.merge(data, immunizations_grouped.add_prefix('IMMUN_'), how='left', left_on=['PATIENT_Id', 'ENCOUNTER_Id'], right_on=['IMMUN_PATIENT', 'IMMUN_ENCOUNTER'])
data = data.drop("IMMUN_PATIENT", axis=1)
data = data.drop("IMMUN_ENCOUNTER", axis=1)


data.head(1)

,PATIENT_Id,PATIENT_BIRTHDATE,PATIENT_DEATHDATE,PATIENT_SSN,PATIENT_FIRST,PATIENT_LAST,PATIENT_MARITAL,PATIENT_RACE,PATIENT_ETHNICITY,PATIENT_GENDER,...,ALLERGY_DESCRIPTION,DEVICE_START,DEVICE_STOP,DEVICE_CODE,DEVICE_DESCRIPTION,DEVICE_UDI,IMMUN_DATE,IMMUN_CODE,IMMUN_DESCRIPTION,IMMUN_BASE_COST
0,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,NaT,999-73-5361,Milo271,Feil794,M,white,nonhispanic,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# sanity check on number of patients according to patients.csv file
len(data["PATIENT_Id"].unique())

1171

In [71]:
data.to_csv("final_data.csv", sep="\t", index=False, encoding="utf-8")